In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [4]:
# Define paths for three datasets
DATASET_PATHS = [
    r'/kaggle/input/cedarbhsig-260/Reconstructed_CEDAR/Reconstructed_CEDAR',
    r'/kaggle/input/cedarbhsig-260/Reconstructed_BHSig100-20250403T193101Z-001/Reconstructed_BHSig100'
    r'/kaggle/input/cedarbhsig-260/Reconstructed_BHSig160-20250403T183033Z-001/Reconstructed_BHSig160'
]

# Preprocessing functions remain unchanged
def preprocess_signature(image_path):
    """Generate two processed images: filtered and grayscale."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None, None

    image = cv2.resize(image, (256, 128))
    filtered = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=1)
    filtered = cv2.GaussianBlur(filtered, (5,5), 0)
    _, filtered = cv2.threshold(filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return filtered, image.copy()

# Modified dataset loader for multiple datasets
def load_multiple_datasets(dataset_paths):
    all_X_train, all_y_train = [], []
    test_sets = []
    i=0
    for path in dataset_paths:
        i=i+1
        images, labels = [], []
        for user_folder in os.listdir(path):
            user_path = os.path.join(path, user_folder)
            if os.path.isdir(user_path):
                for img_name in os.listdir(user_path):
                    img_path = os.path.join(user_path, img_name)
                    filtered, grayscale = preprocess_signature(img_path)
                    if filtered is not None:
                        stacked = np.stack([filtered, grayscale], axis=-1)
                        images.append(stacked)
                        if (i==1):
                          labels.append(0 if "original" in img_name else 1)
                        else :
                          labels.append(0 if "G" in img_name else 1)

        X = np.array(images, dtype=np.float32) / 255.0
        y = np.array(labels)

        # Split dataset into 70-30
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        all_X_train.append(X_train)
        all_y_train.append(y_train)
        test_sets.append((X_test, y_test))

    # Combine training data from all datasets
    X_train_combined = np.concatenate(all_X_train)
    y_train_combined = np.concatenate(all_y_train)

    # Shuffle combined training data
    indices = np.random.permutation(len(X_train_combined))
    return X_train_combined[indices], y_train_combined[indices], test_sets

# Load and split all datasets
X_train, y_train, test_sets = load_multiple_datasets(DATASET_PATHS)

In [2]:
def build_inception_svgnet():
    input_layer = keras.Input(shape=(128, 256, 2))
    
    def inception_module(x, filters):
    # Ensure filters are at least 1
        conv1 = layers.Conv2D(filters, (16,16), padding='same', activation='relu')(x)
        conv2 = layers.Conv2D(filters, (8,8), padding='same', activation='relu')(x)
        conv3 = layers.Conv2D(filters, (4,4), padding='same', activation='relu')(x)
        conv4 = layers.Conv2D(filters, (2,2), padding='same', activation='relu')(x)
        return layers.Concatenate()([conv1, conv2, conv3, conv4])


    x = inception_module(input_layer, 16)
    x = layers.Dropout(0.5)(x)
    x = layers.AveragePooling2D(pool_size=(2,2))(x)
    x = inception_module(x, 24)
    x = layers.Dropout(0.5)(x)
    x = layers.AveragePooling2D(pool_size=(2,2))(x)
    x = inception_module(x, 32)
    x = layers.Dropout(0.5)(x)
    x = layers.AveragePooling2D(pool_size=(2,2))(x)

    x = layers.Conv2D(64, (4,4), activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.AveragePooling2D(pool_size=(2,2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output_layer = layers.Dense(2, activation='softmax')(x)

    model = keras.Model(input_layer, output_layer)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize and train model
model = build_inception_svgnet()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 256, 2)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 256, 16)   │          8,208 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 256, 16)   │          2,064 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 256, 16)   │            528 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 256, 16)   │            144 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 128, 256, 64)   │              0 │ conv2d[0][0],          │
│                           │                        │                │ conv2d_1[0][0],        │
│                           │                        │                │ conv2d_2[0][0],        │
│                           │                        │                │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128, 256, 64)   │              0 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pooling2d         │ (None, 64, 128, 64)    │              0 │ dropout[0][0]          │
│ (AveragePooling2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 128, 24)    │        393,240 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 128, 24)    │         98,328 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 64, 128, 24)    │         24,600 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 64, 128, 24)    │          6,168 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 64, 128, 96)    │              0 │ conv2d_4[0][0],        │
│ (Concatenate)             │                        │                │ conv2d_5[0][0],        │
│                           │                        │                │ conv2d_6[0][0],        │
│                           │                        │                │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64, 128, 96)    │              0 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pooling2d_1       │ (None, 32, 64, 96)     │              0 │ dropout_1[0][0]        │
│ (AveragePooling2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)    

 Total params: 3,151,842 (12.02 MB)

 Trainable params: 3,151,842 (12.02 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.fit(X_train, y_train, validation_split=0.2, epochs=40, batch_size=160)

# Evaluate on each test set individually
for i, (X_test, y_test) in enumerate(test_sets, 1):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Dataset {i} Test Accuracy: {accuracy*100:.2f}%')

Epoch 1/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 348s 4s/step - accuracy: 0.5262 - loss: 0.9929 - val_accuracy: 0.5420 - val_loss: 0.6888
Epoch 2/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.5449 - loss: 0.6893 - val_accuracy: 0.5492 - val_loss: 0.6846
Epoch 3/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.5919 - loss: 0.6630 - val_accuracy: 0.6755 - val_loss: 0.6149
Epoch 4/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.6722 - loss: 0.6044 - val_accuracy: 0.7307 - val_loss: 0.5586
Epoch 5/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.7282 - loss: 0.5562 - val_accuracy: 0.7350 - val_loss: 0.5312
Epoch 6/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.7539 - loss: 0.5142 - val_accuracy: 0.7658 - val_loss: 0.4767
Epoch 7/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.7840 - loss: 0.4681 - val_accuracy: 0.7804 - val_loss: 0.4587
Epoch 8/40
59/59 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.7967 - loss: 0.4441 - val_accuracy: 0.8031 - val_loss